In [61]:
from collections import Counter, defaultdict, namedtuple
from concurrent.futures import ThreadPoolExecutor
from dataclasses import dataclass
from decimal import Decimal
import decimal
from math import sqrt

import diskcache
import polars as pl
import requests
from ape import Contract, chain, convert, networks
from ape.contracts.base import ContractCall, MethodABI
from ape.exceptions import ContractLogicError, VirtualMachineError
from ape.types import AddressType
from ape.utils import ZERO_ADDRESS
from ape_ethereum.multicall import Call
from ethpm_types import ContractType
from ethpm_types.abi import ABIType
from rich import print
from toolz import concat, partition_all, unique, valfilter
from tqdm.notebook import tqdm

In [62]:
cache = diskcache.Cache("cache")

In [63]:
deploy_block = 312549
snapshot_block = 1421059
at_snap = {
    "block_identifier": snapshot_block
}  # use as **at_snap for calls to avoid typos

In [64]:
networks.parse_network_choice("blast:mainnet:geth").__enter__()

<geth chain_id=81457>

In [65]:
chain.provider.concurrency = 2
chain.provider.block_page_size = 10_000

In [66]:
baseline_abi = '{"contractName":"Baseline","abi":[{"type":"constructor","inputs":[{"name":"name_","type":"string","internalType":"string"},{"name":"symbol_","type":"string","internalType":"string"},{"name":"reserve_","type":"address","internalType":"address"},{"name":"salt_","type":"bytes32","internalType":"bytes32"},{"name":"factory_","type":"address","internalType":"address"},{"name":"brs_","type":"address","internalType":"address"}],"stateMutability":"nonpayable"},{"type":"function","name":"FEE_TIER","inputs":[],"outputs":[{"name":"","type":"uint24","internalType":"uint24"}],"stateMutability":"view"},{"type":"function","name":"INTEREST_PER_DIEM","inputs":[],"outputs":[{"name":"","type":"uint256","internalType":"uint256"}],"stateMutability":"view"},{"type":"function","name":"LIQ_FACTOR_GROWTH","inputs":[],"outputs":[{"name":"","type":"uint256","internalType":"uint256"}],"stateMutability":"view"},{"type":"function","name":"PRECISION","inputs":[],"outputs":[{"name":"","type":"uint256","internalType":"uint256"}],"stateMutability":"view"},{"type":"function","name":"REBALANCE_THRESHOLD","inputs":[],"outputs":[{"name":"","type":"int24","internalType":"int24"}],"stateMutability":"view"},{"type":"function","name":"TAKE_RATE","inputs":[],"outputs":[{"name":"","type":"uint256","internalType":"uint256"}],"stateMutability":"view"},{"type":"function","name":"bAsset","inputs":[],"outputs":[{"name":"","type":"address","internalType":"contract BAsset"}],"stateMutability":"view"},{"type":"function","name":"baselineFactory","inputs":[],"outputs":[{"name":"","type":"address","internalType":"contract BaselineFactory"}],"stateMutability":"view"},{"type":"function","name":"borrow","inputs":[{"name":"user_","type":"address","internalType":"address"},{"name":"bAssetsIn_","type":"uint256","internalType":"uint256"},{"name":"numDays_","type":"uint256","internalType":"uint256"}],"outputs":[{"name":"","type":"uint256","internalType":"uint256"},{"name":"","type":"uint256","internalType":"uint256"}],"stateMutability":"nonpayable"},{"type":"function","name":"brs","inputs":[],"outputs":[{"name":"","type":"address","internalType":"address"}],"stateMutability":"view"},{"type":"function","name":"canShift","inputs":[],"outputs":[{"name":"","type":"bool","internalType":"bool"}],"stateMutability":"view"},{"type":"function","name":"canSlide","inputs":[],"outputs":[{"name":"","type":"bool","internalType":"bool"}],"stateMutability":"view"},{"type":"function","name":"checkpointTick","inputs":[],"outputs":[{"name":"","type":"int24","internalType":"int24"}],"stateMutability":"view"},{"type":"function","name":"defaultOutstanding","inputs":[],"outputs":[],"stateMutability":"nonpayable"},{"type":"function","name":"estimateBorrow","inputs":[{"name":"user_","type":"address","internalType":"address"},{"name":"bAssetsIn_","type":"uint256","internalType":"uint256"},{"name":"addDays_","type":"uint256","internalType":"uint256"}],"outputs":[{"name":"estPrincipal","type":"uint256","internalType":"uint256"},{"name":"interest","type":"uint256","internalType":"uint256"},{"name":"newExpiry","type":"uint256","internalType":"uint256"}],"stateMutability":"view"},{"type":"function","name":"floorTick","inputs":[],"outputs":[{"name":"","type":"int24","internalType":"int24"}],"stateMutability":"view"},{"type":"function","name":"getBalancesForPosition","inputs":[{"name":"position_","type":"tuple","internalType":"struct LiquidityManager.Position","components":[{"name":"posType","type":"uint8","internalType":"enum LiquidityManager.PosType"},{"name":"lower","type":"int24","internalType":"int24"},{"name":"upper","type":"int24","internalType":"int24"}]}],"outputs":[{"name":"reserves","type":"uint256","internalType":"uint256"},{"name":"bAssets","type":"uint256","internalType":"uint256"}],"stateMutability":"view"},{"type":"function","name":"getCirculatingSupply","inputs":[],"outputs":[{"name":"","type":"uint256","internalType":"uint256"}],"stateMutability":"view"},{"type":"function","name":"getCreditAccount","inputs":[{"name":"user_","type":"address","internalType":"address"}],"outputs":[{"name":"","type":"tuple","internalType":"struct CreditFacility.CreditAccount","components":[{"name":"principal","type":"uint256","internalType":"uint256"},{"name":"interest","type":"uint256","internalType":"uint256"},{"name":"collateral","type":"uint256","internalType":"uint256"},{"name":"expiry","type":"uint256","internalType":"uint256"},{"name":"lastFloor","type":"uint256","internalType":"uint256"}]}],"stateMutability":"view"},{"type":"function","name":"getFloorPrice","inputs":[],"outputs":[{"name":"","type":"uint256","internalType":"uint256"}],"stateMutability":"view"},{"type":"function","name":"getInterest","inputs":[{"name":"credit_","type":"uint256","internalType":"uint256"},{"name":"numDays_","type":"uint256","internalType":"uint256"}],"outputs":[{"name":"","type":"uint256","internalType":"uint256"}],"stateMutability":"pure"},{"type":"function","name":"getNumDays","inputs":[{"name":"seconds_","type":"uint256","internalType":"uint256"}],"outputs":[{"name":"","type":"uint256","internalType":"uint256"}],"stateMutability":"pure"},{"type":"function","name":"getPosition","inputs":[{"name":"posType_","type":"uint8","internalType":"enum LiquidityManager.PosType"}],"outputs":[{"name":"position","type":"tuple","internalType":"struct LiquidityManager.Position","components":[{"name":"posType","type":"uint8","internalType":"enum LiquidityManager.PosType"},{"name":"lower","type":"int24","internalType":"int24"},{"name":"upper","type":"int24","internalType":"int24"}]}],"stateMutability":"view"},{"type":"function","name":"getPositionLiquidity","inputs":[{"name":"position_","type":"tuple","internalType":"struct LiquidityManager.Position","components":[{"name":"posType","type":"uint8","internalType":"enum LiquidityManager.PosType"},{"name":"lower","type":"int24","internalType":"int24"},{"name":"upper","type":"int24","internalType":"int24"}]}],"outputs":[{"name":"liquidity","type":"uint128","internalType":"uint128"}],"stateMutability":"view"},{"type":"function","name":"getTimeslot","inputs":[{"name":"timestamp_","type":"uint256","internalType":"uint256"}],"outputs":[{"name":"","type":"uint256","internalType":"uint256"}],"stateMutability":"pure"},{"type":"function","name":"init","inputs":[{"name":"preAsset_","type":"address","internalType":"contract PreAsset"},{"name":"router_","type":"address","internalType":"address"},{"name":"initTick_","type":"int24","internalType":"int24"},{"name":"initFloor_","type":"uint256","internalType":"uint256"},{"name":"initDisc_","type":"uint256","internalType":"uint256"}],"outputs":[],"stateMutability":"nonpayable"},{"type":"function","name":"pool","inputs":[],"outputs":[{"name":"","type":"address","internalType":"contract IUniswapV3Pool"}],"stateMutability":"view"},{"type":"function","name":"rebalance","inputs":[],"outputs":[{"name":"didRebalance","type":"bool","internalType":"bool"}],"stateMutability":"nonpayable"},{"type":"function","name":"repay","inputs":[{"name":"user_","type":"address","internalType":"address"},{"name":"reservesIn_","type":"uint256","internalType":"uint256"}],"outputs":[{"name":"","type":"uint256","internalType":"uint256"}],"stateMutability":"nonpayable"},{"type":"function","name":"reserve","inputs":[],"outputs":[{"name":"","type":"address","internalType":"contract ERC20"}],"stateMutability":"view"},{"type":"function","name":"shift","inputs":[],"outputs":[{"name":"","type":"bool","internalType":"bool"}],"stateMutability":"nonpayable"},{"type":"function","name":"slide","inputs":[],"outputs":[{"name":"","type":"bool","internalType":"bool"}],"stateMutability":"nonpayable"},{"type":"function","name":"timeslots","inputs":[{"name":"","type":"uint256","internalType":"uint256"}],"outputs":[{"name":"creditIssued","type":"uint256","internalType":"uint256"},{"name":"defaultableCollateral","type":"uint256","internalType":"uint256"},{"name":"nextTimeslot","type":"uint256","internalType":"uint256"}],"stateMutability":"view"},{"type":"function","name":"totalBAssetCollateral","inputs":[],"outputs":[{"name":"","type":"uint256","internalType":"uint256"}],"stateMutability":"view"},{"type":"function","name":"totalLentReserves","inputs":[],"outputs":[{"name":"","type":"uint256","internalType":"uint256"}],"stateMutability":"view"},{"type":"function","name":"tsHead","inputs":[],"outputs":[{"name":"","type":"uint256","internalType":"uint256"}],"stateMutability":"view"},{"type":"function","name":"tsTail","inputs":[],"outputs":[{"name":"","type":"uint256","internalType":"uint256"}],"stateMutability":"view"},{"type":"function","name":"uniswapV3MintCallback","inputs":[{"name":"tokensOwed_","type":"uint256","internalType":"uint256"},{"name":"reservesOwed_","type":"uint256","internalType":"uint256"},{"name":"","type":"bytes","internalType":"bytes"}],"outputs":[],"stateMutability":"nonpayable"},{"type":"event","name":"Borrow","inputs":[{"name":"user_","type":"address","indexed":true,"internalType":"address"},{"name":"collateralIn_","type":"uint256","indexed":false,"internalType":"uint256"},{"name":"creditOut_","type":"uint256","indexed":false,"internalType":"uint256"},{"name":"interest_","type":"uint256","indexed":false,"internalType":"uint256"}],"anonymous":false},{"type":"event","name":"LiquidityStateSet","inputs":[{"name":"floorTick_","type":"int24","indexed":false,"internalType":"int24"},{"name":"checkpointTick_","type":"int24","indexed":false,"internalType":"int24"}],"anonymous":false},{"type":"event","name":"Repay","inputs":[{"name":"user_","type":"address","indexed":true,"internalType":"address"},{"name":"creditIn_","type":"uint256","indexed":false,"internalType":"uint256"},{"name":"collateralOut_","type":"uint256","indexed":false,"internalType":"uint256"}],"anonymous":false},{"type":"error","name":"InvalidAction","inputs":[]},{"type":"error","name":"InvalidCaller","inputs":[]},{"type":"error","name":"InvalidExpiry","inputs":[]},{"type":"error","name":"InvalidInput","inputs":[]},{"type":"error","name":"InvalidTokenOrder","inputs":[]},{"type":"error","name":"NoCollateralAvailable","inputs":[]},{"type":"error","name":"NoCreditAvailable","inputs":[]},{"type":"error","name":"T","inputs":[]}]}'
basset_abi = '{"contractName":"bAsset","abi":[{"type":"constructor","inputs":[{"name":"name_","type":"string","internalType":"string"},{"name":"symbol_","type":"string","internalType":"string"}],"stateMutability":"nonpayable"},{"type":"function","name":"DOMAIN_SEPARATOR","inputs":[],"outputs":[{"name":"","type":"bytes32","internalType":"bytes32"}],"stateMutability":"view"},{"type":"function","name":"allowance","inputs":[{"name":"","type":"address","internalType":"address"},{"name":"","type":"address","internalType":"address"}],"outputs":[{"name":"","type":"uint256","internalType":"uint256"}],"stateMutability":"view"},{"type":"function","name":"approve","inputs":[{"name":"spender","type":"address","internalType":"address"},{"name":"amount","type":"uint256","internalType":"uint256"}],"outputs":[{"name":"","type":"bool","internalType":"bool"}],"stateMutability":"nonpayable"},{"type":"function","name":"balanceOf","inputs":[{"name":"","type":"address","internalType":"address"}],"outputs":[{"name":"","type":"uint256","internalType":"uint256"}],"stateMutability":"view"},{"type":"function","name":"baseline","inputs":[],"outputs":[{"name":"","type":"address","internalType":"address"}],"stateMutability":"view"},{"type":"function","name":"burn","inputs":[{"name":"amount_","type":"uint256","internalType":"uint256"}],"outputs":[],"stateMutability":"nonpayable"},{"type":"function","name":"decimals","inputs":[],"outputs":[{"name":"","type":"uint8","internalType":"uint8"}],"stateMutability":"view"},{"type":"function","name":"mint","inputs":[{"name":"to","type":"address","internalType":"address"},{"name":"amount_","type":"uint256","internalType":"uint256"}],"outputs":[],"stateMutability":"nonpayable"},{"type":"function","name":"name","inputs":[],"outputs":[{"name":"","type":"string","internalType":"string"}],"stateMutability":"view"},{"type":"function","name":"nonces","inputs":[{"name":"","type":"address","internalType":"address"}],"outputs":[{"name":"","type":"uint256","internalType":"uint256"}],"stateMutability":"view"},{"type":"function","name":"permit","inputs":[{"name":"owner","type":"address","internalType":"address"},{"name":"spender","type":"address","internalType":"address"},{"name":"value","type":"uint256","internalType":"uint256"},{"name":"deadline","type":"uint256","internalType":"uint256"},{"name":"v","type":"uint8","internalType":"uint8"},{"name":"r","type":"bytes32","internalType":"bytes32"},{"name":"s","type":"bytes32","internalType":"bytes32"}],"outputs":[],"stateMutability":"nonpayable"},{"type":"function","name":"symbol","inputs":[],"outputs":[{"name":"","type":"string","internalType":"string"}],"stateMutability":"view"},{"type":"function","name":"totalSupply","inputs":[],"outputs":[{"name":"","type":"uint256","internalType":"uint256"}],"stateMutability":"view"},{"type":"function","name":"transfer","inputs":[{"name":"to","type":"address","internalType":"address"},{"name":"amount","type":"uint256","internalType":"uint256"}],"outputs":[{"name":"","type":"bool","internalType":"bool"}],"stateMutability":"nonpayable"},{"type":"function","name":"transferFrom","inputs":[{"name":"from","type":"address","internalType":"address"},{"name":"to","type":"address","internalType":"address"},{"name":"amount","type":"uint256","internalType":"uint256"}],"outputs":[{"name":"","type":"bool","internalType":"bool"}],"stateMutability":"nonpayable"},{"type":"event","name":"Approval","inputs":[{"name":"owner","type":"address","indexed":true,"internalType":"address"},{"name":"spender","type":"address","indexed":true,"internalType":"address"},{"name":"amount","type":"uint256","indexed":false,"internalType":"uint256"}],"anonymous":false},{"type":"event","name":"Transfer","inputs":[{"name":"from","type":"address","indexed":true,"internalType":"address"},{"name":"to","type":"address","indexed":true,"internalType":"address"},{"name":"amount","type":"uint256","indexed":false,"internalType":"uint256"}],"anonymous":false},{"type":"error","name":"InvalidCaller","inputs":[]}]}'

In [67]:
baseline = Contract(
    "0x14eB8d9b6e19842B5930030B18c50B0391561f27",
    contract_type=ContractType.model_validate_json(baseline_abi),
)
yes = Contract(
    "0x20fE91f17ec9080E3caC2d688b4EcB48C5aC3a9C",
    contract_type=ContractType.model_validate_json(basset_abi),
)
weth = Contract("0x4300000000000000000000000000000000000004")
usdb = Contract("0x4300000000000000000000000000000000000003")

# utils


In [68]:
def transfers_to_balances(logs, zero_address=ZERO_ADDRESS):
    balances = Counter()

    for log in logs:
        src, dst, amt = log.event_arguments.values()
        if src != zero_address:
            balances[src] -= amt
        if dst != zero_address:
            balances[dst] += amt

    return valfilter(bool, dict(balances.most_common()))


def eth(wei, decimals=18):
    return Decimal(wei) / 10**decimals


@dataclass
class Result:
    source: AddressType
    protocol: str
    user: AddressType
    token: AddressType
    balance: Decimal

# spot holders


In [69]:
@cache.memoize()
def get_transfers(token):
    # NOTE in python ranges are [start, end), adding 1 to end is correct
    return list(
        tqdm(
            Contract(token).Transfer.range(deploy_block, snapshot_block + 1),
            desc="logs",
        )
    )

In [70]:
yes_transfers = get_transfers(str(yes))

In [71]:
spot_balances = transfers_to_balances(yes_transfers)
assert sum(spot_balances.values()) == yes.totalSupply(**at_snap), "supply mismatch"
len(spot_balances)

3960

In [72]:
# don't snapshot balances below 1 yes
spot_balances = {k: v for k, v in spot_balances.items() if eth(v) >= 1}
len(spot_balances)

3009

### now we need to filter out contracts so we can manually go through them


In [73]:
spot_addresses = sorted(spot_balances)

In [74]:
@cache.memoize()
@diskcache.throttle(cache, 50, 1)
def get_code(a):
    return chain.provider.get_code(a)

In [75]:
pool = ThreadPoolExecutor(16)
spot_codes = list(tqdm(pool.map(get_code, spot_addresses), "codes"))

codes: 0it [00:00, ?it/s]

In [76]:
spot_contracts = [a for a, b in zip(spot_addresses, spot_codes) if b]

## detectors and unwrappers

- `is_foo(addr) -> bool` detectors return a bool if an address can be attributed to a protocol
- `unwrap_foo(addr) -> Result | list[Result]` unwrappers return one or more `Result` object


### thruster v2 lp


In [77]:
factory_call = MethodABI(
    type="function",
    name="factory",
    stateMutability="view",
    outputs=[ABIType(type="address")],
)
pair_exists_call = MethodABI(
    type="function",
    name="pairExists",
    stateMutability="view",
    inputs=[ABIType(type="address")],
    outputs=[ABIType(type="bool")],
)
decimals_call = MethodABI(
    type="function",
    name="decimals",
    stateMutability="view",
    outputs=[ABIType(type="uint256")],
)
thruster_factories = {
    "0xb4A7D971D0ADea1c73198C97d7ab3f9CE4aaFA13": "0.3%",
    "0x37836821a2c03c171fB1a595767f4a16e2b93Fc4": "1%",
}


@cache.memoize()
def is_thruster_v2_lp(pair):
    try:
        factory = ContractCall(factory_call, pair)(skip_trace=True)
        if factory not in thruster_factories:
            return False
        return ContractCall(pair_exists_call, factory)(pair, skip_trace=True)
    except (ContractLogicError, VirtualMachineError) as e:
        return False


@cache.memoize()
def get_decimals(a):
    if a == ZERO_ADDRESS:
        return 18
    return ContractCall(decimals_call, a)(skip_trace=True)


def unwrap_thruster_v2_lp(a):
    logs = get_transfers(str(a))
    balances = transfers_to_balances(logs)
    pool = Contract(str(a))

    call = Call()
    call.add(pool.token0)
    call.add(pool.token1)
    call.add(pool.getReserves)
    call.add(pool.totalSupply)
    token0, token1, reserves, total_supply = call(**at_snap)
    decimals0 = get_decimals(token0)
    decimals1 = get_decimals(token1)

    results = []
    for u, l in balances.items():
        results.extend(
            [
                Result(
                    source=a,
                    protocol="thruster-v2",
                    user=u,
                    token=token0,
                    balance=eth(l)
                    * eth(reserves._reserve0, decimals0)
                    / eth(total_supply),
                ),
                Result(
                    source=a,
                    protocol="thruster-v2",
                    user=u,
                    token=token1,
                    balance=eth(l)
                    * eth(reserves._reserve1, decimals1)
                    / eth(total_supply),
                ),
            ]
        )
    return results

### safe wallet


In [78]:
master_copy_call = MethodABI(
    type="function",
    name="masterCopy",
    stateMutability="view",
    outputs=[ABIType(type="address")],
)
master_copies = {
    "0xd9Db270c1B5E3Bd161E8c8503c55cEABeE709552": "1.3.0",
    "0x3E5c63644E683549055b9Be8653de26E0B4CD36E": "1.3.0+L2",
}


@cache.memoize()
def is_gnosis_safe(addr):
    try:
        master_copy = ContractCall(master_copy_call, addr)(skip_trace=True)
        return master_copy in master_copies
    except (ContractLogicError, VirtualMachineError) as e:
        return False


def unwrap_gnosis_safe(addr):
    return Result(
        source=addr,
        protocol="safe-wallet",
        user=addr,
        token=str(yes),
        balance=eth(spot_balances[addr]),
    )

### hyperlock


In [79]:
hyperlock = Contract("0xc28EffdfEF75448243c1d9bA972b97e32dF60d06")
hyperlock_deploy = 469338


@cache.memoize()
def get_hyperlock_events():
    return [
        list(
            tqdm(
                hyperlock.Deposit.range(hyperlock_deploy, snapshot_block + 1),
                desc="hyperlock deposits",
            )
        ),
        list(
            tqdm(
                hyperlock.Withdraw.range(hyperlock_deploy, snapshot_block + 1),
                desc="hyperlock withdrawals",
            )
        ),
    ]


hyperlock_deposits, hyperlock_withdrawals = get_hyperlock_events()

In [80]:
hyperlocks = {}  # token_id => {pool, sender}
for log in hyperlock_deposits:
    hyperlocks[log["tokenId"]] = {"pool": log["pool"], "owner": log["sender"]}

### thruster v3


In [81]:
thruster_pos_manager = Contract("0x434575EaEa081b735C985FA9bf63CD7b87e227F9")
thruster_pool_template = Contract("0x7f0DB0D77d0694F29c3f940b5B1F589FFf6EF2e0")
thruster_pool_factory = Contract("0x71b08f13B3c3aF35aAdEb3949AFEb1ded1016127")

In [82]:
@cache.memoize()
def get_pool(token0, token1, fee):
    return thruster_pool_factory.getPool(token0, token1, fee)


@cache.memoize()
def get_slot0(pool):
    return Contract(pool, contract_type=thruster_pool_template.contract_type).slot0(**at_snap).__dict__


@cache.memoize()
def read_nft_position_manager(a):
    pos_manager = Contract(str(a), contract_type=thruster_pos_manager.contract_type)
    num_nft_pos = pos_manager.totalSupply(**at_snap)

    positions = []
    for batch in tqdm(partition_all(1000, range(1, num_nft_pos + 1)), desc="nft positions"):
        call = Call()
        for i in batch:
            call.add(pos_manager.positions, i)

        positions.extend(call(**at_snap))

    # ape bug: returned structs are unpicklable https://github.com/ApeWorX/ape/issues/1976
    positions = [p.__dict__ if p else p for p in positions]

    owners = []
    # need this patch https://github.com/ApeWorX/ape/pull/1975
    # needs batching because of quicknode 5s timeout
    for batch in tqdm(partition_all(1000, range(1, num_nft_pos + 1)), desc="nft owners"):
        call = Call()
        for i in batch:
            call.add(pos_manager.ownerOf, i)

        owners.extend(call(**at_snap))

    return {i: {**p, "owner": o} for i, (p, o) in enumerate(zip(positions, owners), start=1) if p}

In [83]:
# rewritten from https://github.com/Uniswap/v3-periphery/blob/main/contracts/libraries/LiquidityAmounts.sol
def get_sqrt_ratio_at_tick(tick):
    return sqrt(1.0001**tick) * 2**96


def get_amount0_for_liquidity(sqrt_ratio_a_x96, sqrt_ratio_b_x96, liquidity):
    sqrt_ratio_a_x96, sqrt_ratio_b_x96 = sorted([sqrt_ratio_a_x96, sqrt_ratio_b_x96])
    return (
        (liquidity * 2**96)
        * (sqrt_ratio_b_x96 - sqrt_ratio_a_x96)
        // sqrt_ratio_b_x96
        // sqrt_ratio_a_x96
    )


def get_amount1_for_liquidity(sqrt_ratio_a_x96, sqrt_ratio_b_x96, liquidity):
    sqrt_ratio_a_x96, sqrt_ratio_b_x96 = sorted([sqrt_ratio_a_x96, sqrt_ratio_b_x96])
    return liquidity * (sqrt_ratio_b_x96 - sqrt_ratio_a_x96) // 2**96


def get_amounts_for_liquidity(
    sqrt_ratio_x96, sqrt_ratio_a_x96, sqrt_ratio_b_x96, liquidity
):
    sqrt_ratio_a_x96, sqrt_ratio_b_x96 = sorted([sqrt_ratio_a_x96, sqrt_ratio_b_x96])

    if sqrt_ratio_x96 <= sqrt_ratio_a_x96:
        amount0 = get_amount0_for_liquidity(
            sqrt_ratio_a_x96, sqrt_ratio_b_x96, liquidity
        )
        amount1 = 0
    elif sqrt_ratio_x96 < sqrt_ratio_b_x96:
        amount0 = get_amount0_for_liquidity(sqrt_ratio_x96, sqrt_ratio_b_x96, liquidity)
        amount1 = get_amount1_for_liquidity(sqrt_ratio_a_x96, sqrt_ratio_x96, liquidity)
    else:
        amount0 = 0
        amount1 = get_amount1_for_liquidity(
            sqrt_ratio_a_x96, sqrt_ratio_b_x96, liquidity
        )

    return amount0, amount1

In [84]:
pool_exists_call = MethodABI(
    type="function",
    name="poolExists",
    stateMutability="view",
    inputs=[ABIType(type="address")],
    outputs=[ABIType(type="bool")],
)


def is_thruster_v3_pool(pool):
    try:
        factory = ContractCall(factory_call, pool)()
        if factory != str(thruster_pool_factory):
            return False
        return ContractCall(pool_exists_call, factory)(pool)
    except (ContractLogicError, VirtualMachineError):
        return False

In [85]:
particle = Contract("0x121B5ac4De4a3E6F4171956BC26ceda40Cb61a56")


@cache.memoize()
def get_particle_owner(n):
    return particle.lps(n, **at_snap).owner

In [86]:
thruster_positions = read_nft_position_manager(str(thruster_pos_manager))

pool_to_positions = defaultdict(dict)
for i, p in thruster_positions.items():
    if p is None or str(yes) not in [p["token0"], p["token1"]]:
        continue
    pool = get_pool(p["token0"], p["token1"], p["fee"])
    pool_to_positions[pool][i] = p

In [87]:
def unwrap_thruster_v3_nft(i):
    p = thruster_positions[i]

    if p["owner"] == str(
        hyperlock
    ):  # alternatively `i in hyperlocks`, but this is more robust
        pool = hyperlocks[i]["pool"]
        user = hyperlocks[i]["owner"]
        protocol = "hyperlock"
    else:
        pool = get_pool(p["token0"], p["token1"], p["fee"])
        user = p["owner"]
        protocol = "thruster-v3"

    if user == str(particle):
        user = get_particle_owner(i)
        protocol = "particle"

    slot0 = get_slot0(pool)

    amount0, amount1 = get_amounts_for_liquidity(
        slot0["sqrtPriceX96"],
        get_sqrt_ratio_at_tick(p["tickLower"]),
        get_sqrt_ratio_at_tick(p["tickUpper"]),
        p["liquidity"],
    )
    decimals0 = get_decimals(p["token0"])
    decimals1 = get_decimals(p["token1"])

    return [
        Result(
            source=pool,
            protocol=protocol,
            user=user,
            token=p["token0"],
            balance=eth(amount0, decimals0),
        ),
        Result(
            source=pool,
            protocol=protocol,
            user=user,
            token=p["token1"],
            balance=eth(amount1, decimals1),
        ),
    ]


# unwrap_thruster_v3_nft(14030)

In [88]:
def unwrap_thruster_v3_pool(pool):
    results = []
    for i, p in pool_to_positions[pool].items():
        results.extend(unwrap_thruster_v3_nft(i))

    return [x for x in results if x.balance > 0]


# unwrap_thruster_v3_pool('0x07Cfc4c8779d3ab4d6103b3e04F4D719656E81f2')

## baseline


In [89]:
baseline = Contract("0x14eB8d9b6e19842B5930030B18c50B0391561f27")


@cache.memoize()
def get_baseline_credit_accounts():
    borrows = baseline.Borrow.range(312549, snapshot_block + 1)
    users = sorted({x["user_"] for x in tqdm(borrows, desc="baseline borrows")})

    credits = []
    for batch in tqdm(partition_all(500, users), desc="baseline credit accounts"):
        call = Call()
        for user in users:
            call.add(baseline.getCreditAccount, user)
        credits.extend(call(**at_snap))

    return {k: v.__dict__ for k, v in zip(users, credits) if v.expiry != 0}


credit_accounts = get_baseline_credit_accounts()

In [90]:
def is_baseline(a):
    return a in [str(baseline)]


def unwrap_baseline(a):
    results = []
    for user, credit in credit_accounts.items():
        # NOTE the negative weth debt would need to be negated at a later step
        results.extend(
            [
                Result(
                    source=a,
                    protocol="baseline",
                    user=user,
                    token=str(weth),
                    balance=-eth(credit["principal"] + credit["interest"]),
                ),
                Result(
                    source=a,
                    protocol="baseline",
                    user=user,
                    token=str(yes),
                    balance=eth(credit["collateral"]),
                ),
            ]
        )
    return results

### ambient


In [91]:
ambient = Contract("0xaAaaaAAAFfe404EE9433EEf0094b6382D81fb958")
croc_query = Contract("0xA3BD3bE19012De72190c885FB270beb93e36a8A7")

In [92]:
ambient_subgraph = (
    "https://api.studio.thegraph.com/proxy/47610/croc-blast-spare/v0.2.13"
)
# NOTE no pagination needed, there are 295 mint events in the most active pool at the time of writing
ambient_pools_query = """
{
  pools(
    where: {or: [{base: "0x20fE91f17ec9080E3caC2d688b4EcB48C5aC3a9C"}, {quote: "0x20fE91f17ec9080E3caC2d688b4EcB48C5aC3a9C"}]}
  ) {
    id
    base
    quote
    poolIdx
    liquidityChanges(first: 1000, where: {changeType: "mint"}) {
        user
        positionType
        changeType
        bidTick
        askTick
        isBid
    }
  }
}
"""

r = requests.post(ambient_subgraph, json={"query": ambient_pools_query}).json()

In [93]:
AmbientPos = namedtuple(
    "AmbientPos",
    [
        "base",
        "quote",
        "index",
        "user",
        "pos_type",
        "bid_tick",
        "ask_tick",
        "is_bid",
    ],
)
ambient_todo = []

for pool in r["data"]["pools"]:
    for change in pool["liquidityChanges"]:
        ambient_todo.append(
            AmbientPos(
                convert(pool["base"], AddressType),
                convert(pool["quote"], AddressType),
                pool["poolIdx"],
                convert(change["user"], AddressType),
                change["positionType"],
                change["bidTick"],
                change["askTick"],
                change["isBid"],
            )
        )

ambient_todo = list(unique(ambient_todo))
len(ambient_todo)

295

In [94]:
@cache.memoize()
def query_ambient(p: AmbientPos):
    match p.pos_type:
        case "concentrated":
            q = croc_query.queryRangeTokens(
                p.user,
                p.base,
                p.quote,
                p.index,
                p.bid_tick,
                p.ask_tick,
                **at_snap,
            )
        case "knockout":
            pivot_tick = p.bid_tick if p.is_bid else p.ask_tick
            pivot_data = croc_query.queryKnockoutPivot(
                p.base, p.quote, p.index, p.is_bid, pivot_tick, **at_snap
            )
            q = croc_query.queryKnockoutTokens(
                p.user,
                p.base,
                p.quote,
                p.index,
                pivot_data.pivot,
                p.is_bid,
                p.bid_tick,
                p.ask_tick,
                **at_snap,
            )
        case "ambient":
            q = croc_query.queryAmbientTokens(
                p.user, p.base, p.quote, p.index, **at_snap
            )
        case _:
            raise NotImplementedError(p.pos_type)

    decimals_base = get_decimals(p.base)
    decimals_quote = get_decimals(p.quote)
    return [
        Result(
            source=str(ambient),
            protocol="ambient",
            user=p.user,
            token=p.base,
            balance=eth(q.baseQty, decimals_base),
        ),
        Result(
            source=str(ambient),
            protocol="ambient",
            user=p.user,
            token=p.quote,
            balance=eth(q.quoteQty, decimals_quote),
        ),
    ]


def is_ambient(a):
    return a in [str(ambient)]


def unwrap_ambient(a):
    balances = [query_ambient(p) for p in tqdm(ambient_todo, desc="ambient")]
    balances = [x for x in concat(balances) if x.balance > 0]
    return balances


# unwrap_ambient(str(ambient))

### alien


In [95]:
alien = "0x50454acC07bf8fC78100619a1b68e9E8d28cE022"
alien_lens_abi = '[{"type":"function","name":"getMarketMetadata","inputs":[{"type":"address","name":"alien"},{"type":"address","name":"market"}],"outputs":[{"type":"tuple","name":"","components":[{"type":"address","name":"market"},{"type":"string","name":"marketName"},{"type":"string","name":"marketSymbol"},{"type":"uint8","name":"marketDecimals"},{"type":"bool","name":"isListed"},{"type":"uint16","name":"collateralFactor"},{"type":"uint16","name":"liquidationThreshold"},{"type":"uint16","name":"liquidationBonus"},{"type":"uint16","name":"reserveFactor"},{"type":"bool","name":"isPToken"},{"type":"bool","name":"supplyPaused"},{"type":"bool","name":"borrowPaused"},{"type":"bool","name":"transferPaused"},{"type":"bool","name":"isSoftDelisted"},{"type":"address","name":"aTokenAddress"},{"type":"address","name":"debtTokenAddress"},{"type":"address","name":"interestRateModelAddress"},{"type":"uint256","name":"supplyCap"},{"type":"uint256","name":"borrowCap"}]}],"stateMutability":"view"},{"type":"function","name":"getAllMarketsMetadata","inputs":[{"type":"address","name":"alien"}],"outputs":[{"type":"tuple[]","name":"","components":[{"type":"address","name":"market"},{"type":"string","name":"marketName"},{"type":"string","name":"marketSymbol"},{"type":"uint8","name":"marketDecimals"},{"type":"bool","name":"isListed"},{"type":"uint16","name":"collateralFactor"},{"type":"uint16","name":"liquidationThreshold"},{"type":"uint16","name":"liquidationBonus"},{"type":"uint16","name":"reserveFactor"},{"type":"bool","name":"isPToken"},{"type":"bool","name":"supplyPaused"},{"type":"bool","name":"borrowPaused"},{"type":"bool","name":"transferPaused"},{"type":"bool","name":"isSoftDelisted"},{"type":"address","name":"aTokenAddress"},{"type":"address","name":"debtTokenAddress"},{"type":"address","name":"interestRateModelAddress"},{"type":"uint256","name":"supplyCap"},{"type":"uint256","name":"borrowCap"}]}],"stateMutability":"view"},{"type":"function","name":"getCurrentMarketStatus","inputs":[{"type":"address","name":"alien"},{"type":"address","name":"market"}],"outputs":[{"type":"tuple","name":"","components":[{"type":"address","name":"market"},{"type":"uint256","name":"totalCash"},{"type":"uint256","name":"totalBorrow"},{"type":"uint256","name":"totalSupply"},{"type":"uint256","name":"totalReserves"},{"type":"uint256","name":"maxSupplyAmount"},{"type":"uint256","name":"maxBorrowAmount"},{"type":"uint256","name":"marketPrice"},{"type":"uint256","name":"exchangeRate"},{"type":"uint256","name":"supplyRate"},{"type":"uint256","name":"borrowRate"}]}],"stateMutability":"nonpayable"},{"type":"function","name":"getAllMarketsStatus","inputs":[{"type":"address","name":"alien"}],"outputs":[{"type":"tuple[]","name":"","components":[{"type":"address","name":"market"},{"type":"uint256","name":"totalCash"},{"type":"uint256","name":"totalBorrow"},{"type":"uint256","name":"totalSupply"},{"type":"uint256","name":"totalReserves"},{"type":"uint256","name":"maxSupplyAmount"},{"type":"uint256","name":"maxBorrowAmount"},{"type":"uint256","name":"marketPrice"},{"type":"uint256","name":"exchangeRate"},{"type":"uint256","name":"supplyRate"},{"type":"uint256","name":"borrowRate"}]}],"stateMutability":"view"},{"type":"function","name":"getAllCurrentMarketsStatus","inputs":[{"type":"address","name":"alien"}],"outputs":[{"type":"tuple[]","name":"","components":[{"type":"address","name":"market"},{"type":"uint256","name":"totalCash"},{"type":"uint256","name":"totalBorrow"},{"type":"uint256","name":"totalSupply"},{"type":"uint256","name":"totalReserves"},{"type":"uint256","name":"maxSupplyAmount"},{"type":"uint256","name":"maxBorrowAmount"},{"type":"uint256","name":"marketPrice"},{"type":"uint256","name":"exchangeRate"},{"type":"uint256","name":"supplyRate"},{"type":"uint256","name":"borrowRate"}]}],"stateMutability":"nonpayable"},{"type":"function","name":"getUserMarketStatus","inputs":[{"type":"address","name":"alien"},{"type":"address","name":"user"},{"type":"address","name":"market"}],"outputs":[{"type":"tuple","name":"","components":[{"type":"address","name":"market"},{"type":"uint256","name":"balance"},{"type":"uint256","name":"allowanceToAlien"},{"type":"uint256","name":"exchangeRate"},{"type":"uint256","name":"aTokenBalance"},{"type":"uint256","name":"supplyBalance"},{"type":"uint256","name":"borrowBalance"}]}],"stateMutability":"view"},{"type":"function","name":"getCurrentUserMarketStatus","inputs":[{"type":"address","name":"alien"},{"type":"address","name":"user"},{"type":"address","name":"market"}],"outputs":[{"type":"tuple","name":"","components":[{"type":"address","name":"market"},{"type":"uint256","name":"balance"},{"type":"uint256","name":"allowanceToAlien"},{"type":"uint256","name":"exchangeRate"},{"type":"uint256","name":"aTokenBalance"},{"type":"uint256","name":"supplyBalance"},{"type":"uint256","name":"borrowBalance"}]}],"stateMutability":"nonpayable"},{"type":"function","name":"getUserAllMarketsStatus","inputs":[{"type":"address","name":"alien"},{"type":"address","name":"user"}],"outputs":[{"type":"tuple[]","name":"","components":[{"type":"address","name":"market"},{"type":"uint256","name":"balance"},{"type":"uint256","name":"allowanceToAlien"},{"type":"uint256","name":"exchangeRate"},{"type":"uint256","name":"aTokenBalance"},{"type":"uint256","name":"supplyBalance"},{"type":"uint256","name":"borrowBalance"}]}],"stateMutability":"view"},{"type":"function","name":"getUserAllCurrentMarketsStatus","inputs":[{"type":"address","name":"alien"},{"type":"address","name":"user"}],"outputs":[{"type":"tuple[]","name":"","components":[{"type":"address","name":"market"},{"type":"uint256","name":"balance"},{"type":"uint256","name":"allowanceToAlien"},{"type":"uint256","name":"exchangeRate"},{"type":"uint256","name":"aTokenBalance"},{"type":"uint256","name":"supplyBalance"},{"type":"uint256","name":"borrowBalance"}]}],"stateMutability":"nonpayable"}]'
alien_lens = Contract("0xF090b119b10FE4aF048B3EAEB9c0d4821CaBcD30", abi=alien_lens_abi)


def is_alien(a):
    return a in [alien]


def unwrap_alien(a):
    alien_status = alien_lens.getCurrentMarketStatus.call(alien, yes, **at_snap)
    alien_meta = alien_lens.getMarketMetadata(alien, yes, **at_snap)
    alien_yes = Contract(alien_meta.aTokenAddress)
    alien_yes_transfers = get_transfers(str(alien_yes))
    alien_balances = transfers_to_balances(alien_yes_transfers)
    return [
        Result(
            source=alien,
            protocol="alien",
            user=user,
            token=str(yes),
            balance=eth(balance / alien_status.totalSupply * alien_status.totalCash),
        )
        for user, balance in alien_balances.items()
    ]


# unwrap_alien('0x50454acC07bf8fC78100619a1b68e9E8d28cE022')

### ajna


In [96]:
@cache.memoize()
def get_ajna_events(a):
    event_names = [
        "AddQuoteToken",
        "MoveQuoteToken",
        "RemoveQuoteToken",
        "RemoveCollateral",
        "TransferLP",
    ]
    ajna = Contract(a)
    ajna_logs = [
        list(
            tqdm(
                getattr(ajna, event).range(deploy_block, snapshot_block + 1),
                desc="ajna logs",
            )
        )
        for event in event_names
    ]
    ajna_events = sorted(concat(ajna_logs), key=lambda x: (x.block_number, x.log_index))
    return ajna_events

In [97]:
def is_ajna(a):
    return a in ["0x1ca093924a6d719fd9Da27d42fAb83aBdAef6d95"]


def unwrap_ajna(a):
    ajna_events = get_ajna_events(a)
    ajna_lenders = {}
    for e in ajna_events:
        match e.event_name:
            case "AddQuoteToken":
                ajna_lenders[e["lender"]] = {
                    "index": e["index"],
                    "amount": e["amount"],
                    "lp": e["lpAwarded"],
                }
            case "MoveQuoteToken":
                ajna_lenders[e["lender"]] = {
                    "index": e["to"],
                    "amount": e["amount"],
                    "lp": e["lpAwardedTo"],
                }
            case "TransferLP":
                assert (
                    e["lp"] == ajna_lenders[e["owner"]]["lp"]
                ), "partial transfers not implemented"
                ajna_lenders[e["newOwner"]] = ajna_lenders.pop(e["owner"])
            case _:
                raise NotImplementedError()

    cash = spot_balances[a]
    total = sum(x["amount"] for x in ajna_lenders.values())
    return [
        Result(
            source=a,
            protocol="ajna",
            user=user,
            token=str(yes),
            balance=eth(data["amount"] / total * cash),
        )
        for user, data in ajna_lenders.items()
    ]


# unwrap_ajna('0x1ca093924a6d719fd9Da27d42fAb83aBdAef6d95')

### wasabi


In [98]:
wasabi_pools = {
    "0x046299143A880C4d01a318Bc6C9f2C0A5C1Ed355": "WasabiLongPool",
    "0x0301079DaBdC9A2c70b856B2C51ACa02bAc10c3a": "WasabiShortPool",
}


@cache.memoize()
def get_wasabi_logs(a):
    return list(
        tqdm(Contract(a).PositionOpened.range(deploy_block, snapshot_block + 1))
    )


def is_wasabi(a):
    return a in wasabi_pools


def unwrap_wasabi(a):
    is_long = wasabi_pools[a] == "WasabiLongPool"
    logs = get_wasabi_logs(a)
    wasabi = Contract(a)
    currency = "collateralCurrency" if is_long else "currency"
    positions = [x for x in logs if x[currency] == str(yes)]

    call = Call()
    for log in positions:
        call.add(wasabi.positions, log["positionId"])
    hashes = list(call(**at_snap))
    # zero hash means a posititon is inactive
    active_positions = [
        pos for pos, hashed in zip(positions, hashes) if sum(list(hashed))
    ]
    amount_field = "collateralAmount" if is_long else "principal"
    total = sum(p[amount_field] for p in active_positions)
    return [
        Result(
            source=a,
            protocol="wasabi",
            user=p["trader"],
            token=p[currency],
            balance=eth(p[amount_field] / total * spot_balances[a]),
        )
        for p in active_positions
    ]

### generic erc20


In [99]:
def unwrap_erc20(
    addr, protocol="generic_erc20", spot_addr=None, zero_address=ZERO_ADDRESS
):
    if spot_addr is None:
        spot_addr = addr
    token = Contract(addr, contract_type=yes.contract_type)
    transfers = get_transfers(addr)
    balances = transfers_to_balances(transfers, zero_address=zero_address)
    total_supply = token.totalSupply()
    if total_supply != sum(balances.values()):
        print(
            f"warn: supply mismatch. token possibly misses a transfer event on mint/burn. {addr=}"
        )
    return [
        Result(
            source=addr,
            protocol=protocol,
            user=user,
            token=str(yes),
            balance=eth(balance / total_supply * spot_balances[spot_addr]),
        )
        for user, balance in balances.items()
    ]

### roguex


In [100]:
def is_roguex(a):
    return a == "0x30a4D6BB45C96ac1827F6eBCFA6144E5BC0775DC"


def unwrap_roguex(a):
    assert is_roguex(a)
    return unwrap_erc20(
        "0x13b432716e3Ee280BEeBc8F230EdF4e51C7D180B",
        protocol="roguex",
        spot_addr="0x30a4D6BB45C96ac1827F6eBCFA6144E5BC0775DC",
    )

### few


In [101]:
def is_few(a):
    return a == "0x9B96F74a2798C31B7C87A82e4D224951cf514d24"


def unwrap_few(a):
    return unwrap_erc20("0x9B96F74a2798C31B7C87A82e4D224951cf514d24", protocol="few")

### maybe


In [102]:
maybe = "0x4485A08DB1fdbdd6Fea0BF5b8bB95F54e0ff5E9A"


def is_maybe(a):
    return a == maybe


def unwrap_maybe(a):
    return unwrap_erc20(maybe, protocol="maybe", zero_address=maybe)

### unknown contracts with owner


In [103]:
owner_call = MethodABI(
    type="function",
    name="owner",
    stateMutability="view",
    inputs=[],
    outputs=[ABIType(type="address")],
)


@cache.memoize()
def get_owner(addr):
    try:
        owner = ContractCall(owner_call, addr)()
        if owner == ZERO_ADDRESS:
            return None
        return owner
    except (ContractLogicError, VirtualMachineError):
        return None


def unwrap_owner(addr):
    owner = get_owner(addr)
    assert owner is not None
    return Result(
        source=addr,
        protocol="generic_contract_owner",
        user=owner,
        token=str(yes),
        balance=eth(spot_balances[addr]),
    )

## piece it all together


In [104]:
excluded = {
    "0x60BF64CCAa52da304d456892dC0A8f1C5B159f61": "pre_asset (unclaimed yes presale)",
    "0x121B5ac4De4a3E6F4171956BC26ceda40Cb61a56": "particle (unwrapped in thruster v3 nft handler)",
    "0x07964f135f276412b3182a3B2407b8dd45000000": "transitswap router (collects protocol fees)",
    "0x13b432716e3Ee280BEeBc8F230EdF4e51C7D180B": "roguex lp (we use it when unwrapping roguex)",
    "0x20fE91f17ec9080E3caC2d688b4EcB48C5aC3a9C": "yes token (burned)",
    "0x84242b44131378e0D355902f80fbbeC0d4A40eA1": "some rewards distributor",
}
manually_mapped = {
    "0xA635149ecE0FbF3B7cf3453D3c015a1F8c23D2DC": "0x5D58EDC7a7C91239Ec2FD56a646679780886323c",
}


def unwrap_contracts(a):
    if is_thruster_v2_lp(a):
        return unwrap_thruster_v2_lp(a)
    elif is_thruster_v3_pool(a):
        return unwrap_thruster_v3_pool(a)
    elif is_baseline(a):
        return unwrap_baseline(a)
    elif is_ambient(a):
        return unwrap_ambient(a)
    elif is_alien(a):
        return unwrap_alien(a)
    elif is_ajna(a):
        return unwrap_ajna(a)
    elif is_wasabi(a):
        return unwrap_wasabi(a)
    elif is_gnosis_safe(a):
        return unwrap_gnosis_safe(a)
    elif is_roguex(a):
        return unwrap_roguex(a)
    elif is_few(a):
        return unwrap_few(a)
    elif is_maybe(a):
        return unwrap_maybe(a)
    elif a in excluded:
        return []
    elif a in manually_mapped:
        return Result(
            source=a,
            protocol="manually_mapped",
            user=manually_mapped[a],
            token=str(yes),
            balance=eth(spot_balances[a]),
        )
    elif get_owner(a):
        res = unwrap_owner(a)
        print(f"warn: unwrapping {a} to owner {res.user} ({res.balance:,.2f} yes)")
        return res
    else:
        return Result(
            source=a,
            protocol=None,
            user=a,
            token=str(yes),
            balance=eth(spot_balances[a]),
        )

### double unwrap contracts

sometimes when unwrapping things like lps, we end up with contract owners.
this step ensures we identify ultimate beneficiary owners of all of them and leave no token unclaimable.


In [105]:
manually_double_mapped = {
    "0xD40e84a8204aeA3Af36F692F639b195B4750742F": "0x0eA05C8b55DE589F984839D7657da24507720A25",  # few -> ring v2
    "0xA92e68BaaCbFe69de860689b7902adfe9E0F514c": "0xe54c30aBDA52b7d279c05cE9f68782b3871d1F98",  # few -> ring v2
}


def double_unwrap_contracts():
    results = []
    for a in tqdm(spot_contracts):
        res = unwrap_contracts(a)
        if isinstance(res, Result):
            res = []
        for row in res:
            code = get_code(row.user)
            if code and not is_gnosis_safe(row.user):
                owner = get_owner(row.user) or manually_double_mapped.get(row.user)
                if owner:
                    print(f"fixed owner {row.user} -> {owner}")
                    row.user = owner
                else:
                    print("no fix", row)

            results.append(row)

    return results

### team snapshot

In [106]:
TEAM_WALLETS = [
    "0x8044f710c58B6eA6a178CC540f9F1Cd758F7d1B2",
    "0xd271dbA437a363800eb9D8B6479216490BEac97D",
    "0xD7cD8AB6b86aF09a870e0EeAa46665e546c03A70",
    "0x2e742a101818dc93e8677237cECe03A7023D3935",
    "0xbc965428d8d7DBc399EF524Fb6eFa014Dee3afE8",
    "0x36125234DD6c7D0A01dcadC67E79e061D405098e",
    "0xd2FB4d5A11c1e63d910D2A9782d23e63fa369c84",
    "0x816572461D077EBC59165a77e486e9a23495CC9a",
    "0x959aC49B4eebAe995a9b735275fdDae8e11859C7",
    "0xEb1540094eE6c3A7946A45Be686ec9cdF13106a9",
    "0xd1BB2B2871730BC8EF4D86764148C8975b22ce1E",
    "0x37B3492a6F3EfC6A85FCd1C0f221B10EF17Ca76B",
    "0xA984cAD1178972511e18B6B8F4a6BFcE48b9AF62",
    "0x035235999Ac9c80542E7Dcaa22807c8559cb3325",
    "0xEC541890B1b230945112C317125d5D0a3b699c77",
    "0x9C5B9dDBC569A9Bde3D15196E036CBc7A10C50E8",
]


def unwrap_team():
    call = Call()
    for user in TEAM_WALLETS:
        call.add(yes.balanceOf, user)

    # snapshot one block prior to yev operation
    balances = call(block_identifier=snapshot_block - 1)
    team_spot = [
        Result(
            source=user,
            protocol="team-spot",
            user=user,
            token=str(yes),
            balance=eth(balance),
        )
        for user, balance in zip(TEAM_WALLETS, balances)
        if balance > 0
    ]

    # snapshot one block prior to yev operation
    # NOTE no need to exclude elsewhere because these accounts were zeroed out after yev
    call = Call()
    for user in TEAM_WALLETS:
        call.add(baseline.getCreditAccount, user)
    credits = call(block_identifier=snapshot_block - 1)
    team_credit = [
        Result(
            source=user,
            protocol='team-credit',
            user=user,
            token=str(yes),
            balance=eth(credit.collateral),
        )
        for user, credit in zip(TEAM_WALLETS, credits)
        if credit.expiry > 0
    ]
    return team_spot + team_credit

# make the snapshot


In [107]:
def make_snapshot():
    results = []
    for user, balance in spot_balances.items():
        if user in spot_contracts or user in TEAM_WALLETS:
            continue
        results.append(
            Result(
                source=user,
                protocol="spot",
                user=user,
                token=str(yes),
                balance=eth(balance),
            )
        )

    results.extend(double_unwrap_contracts())
    results.extend(unwrap_team())
    return results


snapshot = make_snapshot()

  0%|          | 0/47 [00:00<?, ?it/s]

fixed owner 0xFA50bc65BebDFF536e64e2cb19639abf63a2B43A -> 0x44a8513Ad56d7F094a548aA50C6B6f845b0d90d0

fixed owner 0xFA50bc65BebDFF536e64e2cb19639abf63a2B43A -> 0x44a8513Ad56d7F094a548aA50C6B6f845b0d90d0

warn: unwrapping 0x19813C1F93996e08b2151bEDeDD4f86555dcd401 to owner 0xA942E4d775554B88e7f25f9924b81a861617e6B6 
(20.80 yes)

warn: supply mismatch. token possibly misses a transfer event on mint/burn. 
addr='0x13b432716e3Ee280BEeBc8F230EdF4e51C7D180B'

...

warn: unwrapping 0x662267CaA600d9f284356075C3504E78408A819B to owner 0x8792122Ce00f815b7bCf8e26D0B58eDdc961d35D 
(1.11 yes)

fixed owner 0xD40e84a8204aeA3Af36F692F639b195B4750742F -> 0x0eA05C8b55DE589F984839D7657da24507720A25

fixed owner 0xA92e68BaaCbFe69de860689b7902adfe9E0F514c -> 0xe54c30aBDA52b7d279c05cE9f68782b3871d1F98

warn: unwrapping 0xa177c6c4a36DbBAF97517Ac2FC092b6580aD9e49 to owner 0xFd63Bf84471Bc55DD9A83fdFA293CCBD27e1F4C8 
(2,848.68 yes)

ambient:   0%|          | 0/295 [00:00<?, ?it/s]

In [108]:
pl.Config.set_tbl_rows(20)
pl.Config.set_fmt_str_lengths(50)

df = pl.DataFrame(snapshot, schema_overrides={"balance": pl.Float64})
df.write_csv('snapshot_data.csv')

In [109]:
(
    df.filter(pl.col("token") == str(yes))
    .group_by("protocol")
    .agg(pl.sum("balance"))
    .sort("balance", descending=True)
)

protocol,balance
str,f64
"""baseline""",1.1019e7
"""team-credit""",1.5456e6
"""hyperlock""",1.5316e6
"""spot""",1.3727e6
"""thruster-v3""",175479.609238
"""team-spot""",129533.379031
"""ambient""",95559.435243
"""particle""",62295.102031
"""maybe""",5248.392673
